In [43]:
{
    "tags": [
        "hide-input",
    ]
}

import shelve
import pandas as pd
import numpy as np
import inspect
from IPython.display import display, Markdown

def obtain_input():
    """Obtain user's input with regards to data availability thresholds.

    Formal parameters
    -----------------
    Nil

    Return values
    -------------
    rid_threshold: Integer in the range [0,100]. User-specified data availability threshold for subjects.
    var_threshold: Integer in the range [0,100]. User-specified data availability threshold for variables.
    """
        
    while True:
        try:
            rid_threshold = int(input("Subjects should have >= X% data "
                                      + "availability, where X is: "))
            var_threshold = int(input("Variables should have >= Y% data "
                                      + "availability, where Y is: "))
            if (rid_threshold in range(0,101,1)) and \
            (var_threshold in range(0,101,1)):
                break
            else:
                raise Exception
        except:
            print("Error: Please enter an integer between 0 and 100, inclusive.")
            
    return rid_threshold, var_threshold

def calculate_table_nums(var_names, final_dataset_categories, cat_names):
    """Based on the names of variables available for the user-specified data availability thresholds for subjects and variables,
    calculate the numbers of variables present in each category and sub-category. These numbers will then be displayed in the
    final output table.
    
    Formal parameters
    -----------------
    var_names: List object. Contains the names of variables available for the user-specified data availability thresholds for 
        subjects and variables.
    final_dataset_categories: Pandas dataframe. Contains a list of all variables, and their categories, sub-categories, and
        whether or not they are a key variable.
    cat_names: Pandas dataframe. The index of this dataframe is comprised of all category labels for the variables. It also
        contains the numbers of variables in each category and sub-category, the sub-category labels for each category,
        and the numbers of key variables in each category's subcategory.
        
    Return values
    -------------
    table_nums: Pandas dataframe. Contains the numbers of variables present in each category and sub-category, and the numbers
        of key variables present in each category's sub-category.
    """
    var_names_ref = final_dataset_categories['Variable name']
    rows_selected = np.zeros((len(var_names_ref),1))
    for var_name in var_names:
        rows_selected[np.squeeze(np.where(var_names_ref==var_name)).item()] = 1

    temp_df = final_dataset_categories.iloc[np.squeeze(rows_selected)==1,:]

    # Initialise a results variable for storing the numbers that would then be published in the table. NB: This variable follows
    # the structure of cat_names, not temp_df, because the latter might be smaller.
    table_nums = pd.DataFrame(index=range(0,11), columns=range(0,3))
    table_nums.iloc[1,1] = np.zeros((4,1))
    table_nums.iloc[1,2] = np.zeros((4,1))
    table_nums.iloc[2,1] = np.zeros((9,1))
    table_nums.iloc[2,2] = np.zeros((9,1))
    table_nums.iloc[3,1] = np.zeros((4,1))
    table_nums.iloc[3,2] = np.zeros((4,1))
    table_nums.iloc[4,1] = np.zeros((7,1))
    table_nums.iloc[4,2] = np.zeros((7,1))
    table_nums.iloc[5,1] = np.zeros((3,1))
    table_nums.iloc[5,2] = np.zeros((3,1))
    table_nums.iloc[6,1] = np.zeros((10,1))
    table_nums.iloc[6,2] = np.zeros((10,1))
    table_nums.iloc[7,1] = np.zeros((23,1))
    table_nums.iloc[7,2] = np.zeros((23,1))
    table_nums.iloc[8,1] = np.zeros((2,1))
    table_nums.iloc[8,2] = np.zeros((2,1))
    table_nums.fillna(0, inplace=True)

    for cat_idx, cat_name in enumerate(cat_names.index):

        temp_cat_names = np.unique(temp_df['Category of variable'])

        if cat_name in temp_cat_names:
            temp_df_subset = temp_df.loc[temp_df['Category of variable']==cat_name,:]
            table_nums.iloc[cat_idx,0] = temp_df_subset.shape[0]

            temp_subcat_names = np.unique(temp_df_subset['Sub-category of variable'])
            subcat_names_ref = cat_names.subcat_names[cat_idx]

            for subcat_idx, subcat_name in enumerate(subcat_names_ref):
                if subcat_name in temp_subcat_names:
                    rows_selected = temp_df_subset['Sub-category of variable']==subcat_name
                    try: # For categories that have > 1 sub-category.
                        table_nums.iloc[cat_idx,1][subcat_idx] = np.sum(rows_selected)
                        table_nums.iloc[cat_idx,2][subcat_idx] = np.sum(temp_df_subset.loc[rows_selected, 'Key variable'])
                    except: # For categories that only have 1 sub-category, chained indexing >1 time does not work.
                        table_nums.iloc[cat_idx,1] = np.sum(rows_selected)
                        table_nums.iloc[cat_idx,2] = np.sum(temp_df_subset.loc[rows_selected, 'Key variable'])

    table_nums.iloc[10,0] = np.sum(table_nums.iloc[:9,0])

    for row in table_nums.index:
        if row < 10:
            table_nums.iloc[10,2] += np.sum(table_nums.iloc[row,2])
    
    return table_nums

def copy_locals():
    """Copies all local variables from this context into Jupyter notebook's top level.
        
    This function was copied from https://gist.github.com/orm011/4044aff776f6cbd0cf5c7c3236abc9ed, and then modified slightly.
    """
    stack = inspect.stack()
    
    caller = stack[1]
    local_dict = {k:v  for k,v in caller.frame.f_locals.items() if not k.startswith('_')}
    
    notebook_caller = None
    for st in stack:
        if st.function == '<module>':        
            notebook_caller = st
            break
            
    if notebook_caller is None:
        print('is this being called from within a jupyter notebook?')
        return
    
    #breakpoint()
    #print('copying variables to <module> globals...', list(local_dict.keys()))
    notebook_caller.frame.f_globals.update(local_dict)

def main():
    """Main function for running the program.

    Formal parameters
    -----------------
    Nil

    Return values
    -------------
    Nil
    """
    
    # Import and calculate data.
    with shelve.open('data_availability_draft_1') as db:
        #results_rid_num = db['results_rid_num'] # Numpy array
        #results_var_num = db['results_var_num'] # Numpy array
        results_var_names = db['results_var_names'] # Pandas dataframe
        #results_data_availability = db['results_data_availability'] # Numpy array
        cat_names = db['cat_names'] # Pandas dataframe

    final_dataset_categories = pd.read_csv('final_dataset_categories.csv')

    total_key_var_num = 0
    for array in cat_names.key_var_num:
        total_key_var_num += np.sum(array)
    
    rid_threshold, var_threshold = obtain_input()
    var_names = results_var_names.iloc[rid_threshold, var_threshold]
    table_nums = calculate_table_nums(var_names, final_dataset_categories, cat_names)
    copy_locals()
    print("End of program. Re-run this cell then the next cell if you want to generate a table with different inputs.")

if __name__ == "__main__":
    main()

Subjects should have >= X% data availability, where X is: 46
Variables should have >= Y% data availability, where Y is: 21
End of program. Re-run this cell then the next cell if you want to generate a table with different inputs.


In [44]:
{
    "tags": [
        "hide-input",
    ]
}

output = Markdown("""
Brackets contain numbers in this format: ( Number of variables included / Total number of variables in this category or sub-category )<br><br>
(NB: The column 'Names of variables' has been left blank for now because there are too many variables, and because Markdown automatically wraps the text - I was not able to figure out how to disable this function - each row becomes very tall and renders the table unreadable.)

Category | Sub-category | Key variables | Names of variables
---|---|---|---
{0} ({1} / {2}) | {3} ({4} / {5}) | ({6} / {7}) | {8}
{9} ({10} / {11}) | {12} ({13} / {14}) | ({15} / {16}) | {17}
{18} ({19} / {20}) | {21} ({22} / {23}) | ({24} / {25}) | {26}
{27} ({28} / {29}) | {30} ({31} / {32}) | ({33} / {34}) | {35}
{36} ({37} / {38}) | {39} ({40} / {41}) | ({42} / {43}) | {44}
{45} ({46} / {47}) | {48} ({49} / {50}) | ({51} / {52}) | {53}
{54} ({55} / {56}) | {57} ({58} / {59}) | ({60} / {61}) | {62}
{63} ({64} / {65}) | {66} ({67} / {68}) | ({69} / {70}) | {71}
{72} ({73} / {74}) | {75} ({76} / {77}) | ({78} / {79}) | {80}
{81} ({82} / {83}) | {84} ({85} / {86}) | ({87} / {88}) | {89}
{90} ({91} / {92}) | {93} ({94} / {95}) | ({96} / {97}) | {98}
{99} ({100} / {101}) | {102} ({103} / {104}) | ({105} / {106}) | {107}
{108} ({109} / {110}) | {111} ({112} / {113}) | ({114} / {115}) | {116}
{117} ({118} / {119}) | {120} ({121} / {122}) | ({123} / {124}) | {125}
{126} ({127} / {128}) | {129} ({130} / {131}) | ({132} / {133}) | {134}
{135} ({136} / {137}) | {138} ({139} / {140}) | ({141} / {142}) | {143}
{144} ({145} / {146}) | {147} ({148} / {149}) | ({150} / {151}) | {152}
{153} ({154} / {155}) | {156} ({157} / {158}) | ({159} / {160}) | {161}
{162} ({163} / {164}) | {165} ({166} / {167}) | ({168} / {169}) | {170}
{171} ({172} / {173}) | {174} ({175} / {176}) | ({177} / {178}) | {179}
{180} ({181} / {182}) | {183} ({184} / {185}) | ({186} / {187}) | {188}
{189} ({190} / {191}) | {192} ({193} / {194}) | ({195} / {196}) | {197}
{198} ({199} / {200}) | {201} ({202} / {203}) | ({204} / {205}) | {206}
{207} ({208} / {209}) | {210} ({211} / {212}) | ({213} / {214}) | {215}
{216} ({217} / {218}) | {219} ({220} / {221}) | ({222} / {223}) | {224}
{225} ({226} / {227}) | {228} ({229} / {230}) | ({231} / {232}) | {233}
{234} ({235} / {236}) | {237} ({238} / {239}) | ({240} / {241}) | {242}
{243} ({244} / {245}) | {246} ({247} / {248}) | ({249} / {250}) | {251}
{252} ({253} / {254}) | {255} ({256} / {257}) | ({258} / {259}) | {260}
{261} ({262} / {263}) | {264} ({265} / {266}) | ({267} / {268}) | {269}
{270} ({271} / {272}) | {273} ({274} / {275}) | ({276} / {277}) | {278}
{279} ({280} / {281}) | {282} ({283} / {284}) | ({285} / {286}) | {287}
{288} ({289} / {290}) | {291} ({292} / {293}) | ({294} / {295}) | {296}
{297} ({298} / {299}) | {300} ({301} / {302}) | ({303} / {304}) | {305}
{306} ({307} / {308}) | {309} ({310} / {311}) | ({312} / {313}) | {314}
{315} ({316} / {317}) | {318} ({319} / {320}) | ({321} / {322}) | {323}
{324} ({325} / {326}) | {327} ({328} / {329}) | ({330} / {331}) | {332}
{333} ({334} / {335}) | {336} ({337} / {338}) | ({339} / {340}) | {341}
{342} ({343} / {344}) | {345} ({346} / {347}) | ({348} / {349}) | {350}
{351} ({352} / {353}) | {354} ({355} / {356}) | ({357} / {358}) | {359}
{360} ({361} / {362}) | {363} ({364} / {365}) | ({366} / {367}) | {368}
{369} ({370} / {371}) | {372} ({373} / {374}) | ({375} / {376}) | {377}
{378} ({379} / {380}) | {381} ({382} / {383}) | ({384} / {385}) | {386}
{387} ({388} / {389}) | {390} ({391} / {392}) | ({393} / {394}) | {395}
{396} ({397} / {398}) | {399} ({400} / {401}) | ({402} / {403}) | {404}
{405} ({406} / {407}) | {408} ({409} / {410}) | ({411} / {412}) | {413}
{414} ({415} / {416}) | {417} ({418} / {419}) | ({420} / {421}) | {422}
{423} ({424} / {425}) | {426} ({427} / {428}) | ({429} / {430}) | {431}
{432} ({433} / {434}) | {435} ({436} / {437}) | ({438} / {439}) | {440}
{441} ({442} / {443}) | {444} ({445} / {446}) | ({447} / {448}) | {449}
{450} ({451} / {452}) | {453} ({454} / {455}) | ({456} / {457}) | {458}
{459} ({460} / {461}) | {462} ({463} / {464}) | ({465} / {466}) | {467}
{468} ({469} / {470}) | {471} ({472} / {473}) | ({474} / {475}) | {476}
{477} ({478} / {479}) | {480} ({481} / {482}) | ({483} / {484}) | {485}
{486} ({487} / {488}) | {489} ({490} / {491}) | ({492} / {493}) | {494}
{495} ({496} / {497}) | {498} ({499} / {500}) | ({501} / {502}) | {503}
{504} ({505} / {506}) | {507} ({508} / {509}) | ({510} / {511}) | {512}
{513} ({514} / {515}) | {516} ({517} / {518}) | ({519} / {520}) | {521}
{522} ({523} / {524}) | {525} ({526} / {527}) | ({528} / {529}) | {530}
{531} ({532} / {533}) | {534} ({535} / {536}) | ({537} / {538}) | {539}
{540} ({541} / {542}) | {543} ({544} / {545}) | ({546} / {547}) | {548}
{549} ({550} / {551}) | {552} ({553} / {554}) | ({555} / {556}) | {557}
{558} ({559} / {560}) | {561} ({562} / {563}) | ({564} / {565}) | {566}
{567} ({568} / {569}) | {570} ({571} / {572}) | ({573} / {574}) | {575}
{576} ({577} / {578}) | {579} ({580} / {581}) | ({582} / {583}) | {584}
""".format(
    cat_names.index[0], table_nums.iloc[0,0], cat_names.cat_var_num[0], cat_names.subcat_names[0][0], table_nums.iloc[0,1], int(cat_names.subcat_var_num[0].item()), int(table_nums.iloc[0,2]), int(cat_names.key_var_num[0].item()), "",
    
    cat_names.index[1], table_nums.iloc[1,0], cat_names.cat_var_num[1], cat_names.subcat_names[1][0], int(table_nums.iloc[1,1][0].item()), int(cat_names.subcat_var_num[1][0].item()), int(table_nums.iloc[1,2][0].item()), int(cat_names.key_var_num[1][0].item()), "",
    "", "", "", cat_names.subcat_names[1][1], int(table_nums.iloc[1,1][1].item()), int(cat_names.subcat_var_num[1][1].item()), int(table_nums.iloc[1,2][1].item()), int(cat_names.key_var_num[1][1].item()), "",
    "", "", "", cat_names.subcat_names[1][2], int(table_nums.iloc[1,1][2].item()), int(cat_names.subcat_var_num[1][2].item()), int(table_nums.iloc[1,2][2].item()), int(cat_names.key_var_num[1][2].item()), "",
    "", "", "", cat_names.subcat_names[1][3], int(table_nums.iloc[1,1][3].item()), int(cat_names.subcat_var_num[1][3].item()), int(table_nums.iloc[1,2][3].item()), int(cat_names.key_var_num[1][3].item()), "",
    
    cat_names.index[2], table_nums.iloc[2,0], cat_names.cat_var_num[2], cat_names.subcat_names[2][0], int(table_nums.iloc[2,1][0].item()), int(cat_names.subcat_var_num[2][0].item()), int(table_nums.iloc[2,2][0].item()), int(cat_names.key_var_num[2][0].item()), "",
    "", "", "", cat_names.subcat_names[2][1], int(table_nums.iloc[2,1][1].item()), int(cat_names.subcat_var_num[2][1].item()), int(table_nums.iloc[2,2][1].item()), int(cat_names.key_var_num[2][1].item()), "",
    "", "", "", cat_names.subcat_names[2][2], int(table_nums.iloc[2,1][2].item()), int(cat_names.subcat_var_num[2][2].item()), int(table_nums.iloc[2,2][2].item()), int(cat_names.key_var_num[2][2].item()), "",
    "", "", "", cat_names.subcat_names[2][3], int(table_nums.iloc[2,1][3].item()), int(cat_names.subcat_var_num[2][3].item()), int(table_nums.iloc[2,2][3].item()), int(cat_names.key_var_num[2][3].item()), "",
    "", "", "", cat_names.subcat_names[2][4], int(table_nums.iloc[2,1][4].item()), int(cat_names.subcat_var_num[2][4].item()), int(table_nums.iloc[2,2][4].item()), int(cat_names.key_var_num[2][4].item()), "",
    "", "", "", cat_names.subcat_names[2][5], int(table_nums.iloc[2,1][5].item()), int(cat_names.subcat_var_num[2][5].item()), int(table_nums.iloc[2,2][5].item()), int(cat_names.key_var_num[2][5].item()), "",
    "", "", "", cat_names.subcat_names[2][6], int(table_nums.iloc[2,1][6].item()), int(cat_names.subcat_var_num[2][6].item()), int(table_nums.iloc[2,2][6].item()), int(cat_names.key_var_num[2][6].item()), "",
    "", "", "", cat_names.subcat_names[2][7], int(table_nums.iloc[2,1][7].item()), int(cat_names.subcat_var_num[2][7].item()), int(table_nums.iloc[2,2][7].item()), int(cat_names.key_var_num[2][7].item()), "",
    "", "", "", cat_names.subcat_names[2][8], int(table_nums.iloc[2,1][8].item()), int(cat_names.subcat_var_num[2][8].item()), int(table_nums.iloc[2,2][8].item()), int(cat_names.key_var_num[2][8].item()), "",
    
    cat_names.index[3], table_nums.iloc[3,0], cat_names.cat_var_num[3], cat_names.subcat_names[3][0], int(table_nums.iloc[3,1][0].item()), int(cat_names.subcat_var_num[3][0].item()), int(table_nums.iloc[3,2][0].item()), int(cat_names.key_var_num[3][0].item()), "",
    "", "", "", cat_names.subcat_names[3][1], int(table_nums.iloc[3,1][1].item()), int(cat_names.subcat_var_num[3][1].item()), int(table_nums.iloc[3,2][1].item()), int(cat_names.key_var_num[3][1].item()), "",
    "", "", "", cat_names.subcat_names[3][2], int(table_nums.iloc[3,1][2].item()), int(cat_names.subcat_var_num[3][2].item()), int(table_nums.iloc[3,2][2].item()), int(cat_names.key_var_num[3][2].item()), "",
    "", "", "", cat_names.subcat_names[3][3], int(table_nums.iloc[3,1][3].item()), int(cat_names.subcat_var_num[3][3].item()), int(table_nums.iloc[3,2][3].item()), int(cat_names.key_var_num[3][3].item()), "",
    
    cat_names.index[4], table_nums.iloc[4,0], cat_names.cat_var_num[4], cat_names.subcat_names[4][0], int(table_nums.iloc[4,1][0].item()), int(cat_names.subcat_var_num[4][0].item()), int(table_nums.iloc[4,2][0].item()), int(cat_names.key_var_num[4][0].item()), "",
    "", "", "", cat_names.subcat_names[4][1], int(table_nums.iloc[4,1][1].item()), int(cat_names.subcat_var_num[4][1].item()), int(table_nums.iloc[4,2][1].item()), int(cat_names.key_var_num[4][1].item()), "",
    "", "", "", cat_names.subcat_names[4][2], int(table_nums.iloc[4,1][2].item()), int(cat_names.subcat_var_num[4][2].item()), int(table_nums.iloc[4,2][2].item()), int(cat_names.key_var_num[4][2].item()), "",
    "", "", "", cat_names.subcat_names[4][3], int(table_nums.iloc[4,1][3].item()), int(cat_names.subcat_var_num[4][3].item()), int(table_nums.iloc[4,2][3].item()), int(cat_names.key_var_num[4][3].item()), "",
    "", "", "", cat_names.subcat_names[4][4], int(table_nums.iloc[4,1][4].item()), int(cat_names.subcat_var_num[4][4].item()), int(table_nums.iloc[4,2][4].item()), int(cat_names.key_var_num[4][4].item()), "",
    "", "", "", cat_names.subcat_names[4][5], int(table_nums.iloc[4,1][5].item()), int(cat_names.subcat_var_num[4][5].item()), int(table_nums.iloc[4,2][5].item()), int(cat_names.key_var_num[4][5].item()), "",
    "", "", "", cat_names.subcat_names[4][6], int(table_nums.iloc[4,1][6].item()), int(cat_names.subcat_var_num[4][6].item()), int(table_nums.iloc[4,2][6].item()), int(cat_names.key_var_num[4][6].item()), "",
    
    cat_names.index[5], table_nums.iloc[5,0], cat_names.cat_var_num[5], cat_names.subcat_names[5][0], int(table_nums.iloc[5,1][0].item()), int(cat_names.subcat_var_num[5][0].item()), int(table_nums.iloc[5,2][0].item()), int(cat_names.key_var_num[5][0].item()), "",
    "", "", "", cat_names.subcat_names[5][1], int(table_nums.iloc[5,1][1].item()), int(cat_names.subcat_var_num[5][1].item()), int(table_nums.iloc[5,2][1].item()), int(cat_names.key_var_num[5][1].item()), "",
    "", "", "", cat_names.subcat_names[5][2], int(table_nums.iloc[5,1][2].item()), int(cat_names.subcat_var_num[5][2].item()), int(table_nums.iloc[5,2][2].item()), int(cat_names.key_var_num[5][2].item()), "",
    
    cat_names.index[6], table_nums.iloc[6,0], cat_names.cat_var_num[6], cat_names.subcat_names[6][0], int(table_nums.iloc[6,1][0].item()), int(cat_names.subcat_var_num[6][0].item()), int(table_nums.iloc[6,2][0].item()), int(cat_names.key_var_num[6][0].item()), "",
    "", "", "", cat_names.subcat_names[6][1], int(table_nums.iloc[6,1][1].item()), int(cat_names.subcat_var_num[6][1].item()), int(table_nums.iloc[6,2][1].item()), int(cat_names.key_var_num[6][1].item()), "",
    "", "", "", cat_names.subcat_names[6][2], int(table_nums.iloc[6,1][2].item()), int(cat_names.subcat_var_num[6][2].item()), int(table_nums.iloc[6,2][2].item()), int(cat_names.key_var_num[6][2].item()), "",
    "", "", "", cat_names.subcat_names[6][3], int(table_nums.iloc[6,1][3].item()), int(cat_names.subcat_var_num[6][3].item()), int(table_nums.iloc[6,2][3].item()), int(cat_names.key_var_num[6][3].item()), "",
    "", "", "", cat_names.subcat_names[6][4], int(table_nums.iloc[6,1][4].item()), int(cat_names.subcat_var_num[6][4].item()), int(table_nums.iloc[6,2][4].item()), int(cat_names.key_var_num[6][4].item()), "",
    "", "", "", cat_names.subcat_names[6][5], int(table_nums.iloc[6,1][5].item()), int(cat_names.subcat_var_num[6][5].item()), int(table_nums.iloc[6,2][5].item()), int(cat_names.key_var_num[6][5].item()), "",
    "", "", "", cat_names.subcat_names[6][6], int(table_nums.iloc[6,1][6].item()), int(cat_names.subcat_var_num[6][6].item()), int(table_nums.iloc[6,2][6].item()), int(cat_names.key_var_num[6][6].item()), "",
    "", "", "", cat_names.subcat_names[6][7], int(table_nums.iloc[6,1][7].item()), int(cat_names.subcat_var_num[6][7].item()), int(table_nums.iloc[6,2][7].item()), int(cat_names.key_var_num[6][7].item()), "",
    "", "", "", cat_names.subcat_names[6][8], int(table_nums.iloc[6,1][8].item()), int(cat_names.subcat_var_num[6][8].item()), int(table_nums.iloc[6,2][8].item()), int(cat_names.key_var_num[6][8].item()), "",
    "", "", "", cat_names.subcat_names[6][9], int(table_nums.iloc[6,1][9].item()), int(cat_names.subcat_var_num[6][9].item()), int(table_nums.iloc[6,2][9].item()), int(cat_names.key_var_num[6][9].item()), "",
    
    cat_names.index[7], table_nums.iloc[7,0], cat_names.cat_var_num[7], cat_names.subcat_names[7][0], int(table_nums.iloc[7,1][0].item()), int(cat_names.subcat_var_num[7][0].item()), int(table_nums.iloc[7,2][0].item()), int(cat_names.key_var_num[7][0].item()), "",
    "", "", "", cat_names.subcat_names[7][1], int(table_nums.iloc[7,1][1].item()), int(cat_names.subcat_var_num[7][1].item()), int(table_nums.iloc[7,2][1].item()), int(cat_names.key_var_num[7][1].item()), "",
    "", "", "", cat_names.subcat_names[7][2], int(table_nums.iloc[7,1][2].item()), int(cat_names.subcat_var_num[7][2].item()), int(table_nums.iloc[7,2][2].item()), int(cat_names.key_var_num[7][2].item()), "",
    "", "", "", cat_names.subcat_names[7][3], int(table_nums.iloc[7,1][3].item()), int(cat_names.subcat_var_num[7][3].item()), int(table_nums.iloc[7,2][3].item()), int(cat_names.key_var_num[7][3].item()), "",
    "", "", "", cat_names.subcat_names[7][4], int(table_nums.iloc[7,1][4].item()), int(cat_names.subcat_var_num[7][4].item()), int(table_nums.iloc[7,2][4].item()), int(cat_names.key_var_num[7][4].item()), "",
    "", "", "", cat_names.subcat_names[7][5], int(table_nums.iloc[7,1][5].item()), int(cat_names.subcat_var_num[7][5].item()), int(table_nums.iloc[7,2][5].item()), int(cat_names.key_var_num[7][5].item()), "",
    "", "", "", cat_names.subcat_names[7][6], int(table_nums.iloc[7,1][6].item()), int(cat_names.subcat_var_num[7][6].item()), int(table_nums.iloc[7,2][6].item()), int(cat_names.key_var_num[7][6].item()), "",
    "", "", "", cat_names.subcat_names[7][7], int(table_nums.iloc[7,1][7].item()), int(cat_names.subcat_var_num[7][7].item()), int(table_nums.iloc[7,2][7].item()), int(cat_names.key_var_num[7][7].item()), "",
    "", "", "", cat_names.subcat_names[7][8], int(table_nums.iloc[7,1][8].item()), int(cat_names.subcat_var_num[7][8].item()), int(table_nums.iloc[7,2][8].item()), int(cat_names.key_var_num[7][8].item()), "",
    "", "", "", cat_names.subcat_names[7][9], int(table_nums.iloc[7,1][9].item()), int(cat_names.subcat_var_num[7][9].item()), int(table_nums.iloc[7,2][9].item()), int(cat_names.key_var_num[7][9].item()), "",
    "", "", "", cat_names.subcat_names[7][10], int(table_nums.iloc[7,1][10].item()), int(cat_names.subcat_var_num[7][10].item()), int(table_nums.iloc[7,2][10].item()), int(cat_names.key_var_num[7][10].item()), "",
    "", "", "", cat_names.subcat_names[7][11], int(table_nums.iloc[7,1][11].item()), int(cat_names.subcat_var_num[7][11].item()), int(table_nums.iloc[7,2][11].item()), int(cat_names.key_var_num[7][11].item()), "",
    "", "", "", cat_names.subcat_names[7][12], int(table_nums.iloc[7,1][12].item()), int(cat_names.subcat_var_num[7][12].item()), int(table_nums.iloc[7,2][12].item()), int(cat_names.key_var_num[7][12].item()), "",
    "", "", "", cat_names.subcat_names[7][13], int(table_nums.iloc[7,1][13].item()), int(cat_names.subcat_var_num[7][13].item()), int(table_nums.iloc[7,2][13].item()), int(cat_names.key_var_num[7][13].item()), "",
    "", "", "", cat_names.subcat_names[7][14], int(table_nums.iloc[7,1][14].item()), int(cat_names.subcat_var_num[7][14].item()), int(table_nums.iloc[7,2][14].item()), int(cat_names.key_var_num[7][14].item()), "",
    "", "", "", cat_names.subcat_names[7][15], int(table_nums.iloc[7,1][15].item()), int(cat_names.subcat_var_num[7][15].item()), int(table_nums.iloc[7,2][15].item()), int(cat_names.key_var_num[7][15].item()), "",
    "", "", "", cat_names.subcat_names[7][16], int(table_nums.iloc[7,1][16].item()), int(cat_names.subcat_var_num[7][16].item()), int(table_nums.iloc[7,2][16].item()), int(cat_names.key_var_num[7][16].item()), "",
    "", "", "", cat_names.subcat_names[7][17], int(table_nums.iloc[7,1][17].item()), int(cat_names.subcat_var_num[7][17].item()), int(table_nums.iloc[7,2][17].item()), int(cat_names.key_var_num[7][17].item()), "",
    "", "", "", cat_names.subcat_names[7][18], int(table_nums.iloc[7,1][18].item()), int(cat_names.subcat_var_num[7][18].item()), int(table_nums.iloc[7,2][18].item()), int(cat_names.key_var_num[7][18].item()), "",
    "", "", "", cat_names.subcat_names[7][19], int(table_nums.iloc[7,1][19].item()), int(cat_names.subcat_var_num[7][19].item()), int(table_nums.iloc[7,2][19].item()), int(cat_names.key_var_num[7][19].item()), "",
    "", "", "", cat_names.subcat_names[7][20], int(table_nums.iloc[7,1][20].item()), int(cat_names.subcat_var_num[7][20].item()), int(table_nums.iloc[7,2][20].item()), int(cat_names.key_var_num[7][20].item()), "",
    "", "", "", cat_names.subcat_names[7][21], int(table_nums.iloc[7,1][21].item()), int(cat_names.subcat_var_num[7][21].item()), int(table_nums.iloc[7,2][21].item()), int(cat_names.key_var_num[7][21].item()), "",
    "", "", "", cat_names.subcat_names[7][22], int(table_nums.iloc[7,1][22].item()), int(cat_names.subcat_var_num[7][22].item()), int(table_nums.iloc[7,2][22].item()), int(cat_names.key_var_num[7][22].item()), "",
    
    cat_names.index[8], table_nums.iloc[8,0], cat_names.cat_var_num[8], cat_names.subcat_names[8][0], int(table_nums.iloc[8,1][0].item()), int(cat_names.subcat_var_num[8][0].item()), int(table_nums.iloc[8,2][0].item()), int(cat_names.key_var_num[8][0].item()), "",
    "", "", "", cat_names.subcat_names[8][1], int(table_nums.iloc[8,1][1].item()), int(cat_names.subcat_var_num[8][1].item()), int(table_nums.iloc[8,2][1].item()), int(cat_names.key_var_num[8][1].item()), "",
    
    cat_names.index[9], table_nums.iloc[9,0], cat_names.cat_var_num[9], cat_names.subcat_names[9][0], table_nums.iloc[9,1], int(cat_names.subcat_var_num[9].item()), int(table_nums.iloc[9,2]), int(cat_names.key_var_num[9].item()), "",
    
    "TOTAL", table_nums.iloc[10,0], np.sum(cat_names.cat_var_num), "", "", "", int(table_nums.iloc[10,2]), int(total_key_var_num), ""
)
)
output


Brackets contain numbers in this format: ( Number of variables included / Total number of variables in this category or sub-category )<br><br>
(NB: The column 'Names of variables' has been left blank for now because there are too many variables, and because Markdown automatically wraps the text - I was not able to figure out how to disable this function - each row becomes very tall and renders the table unreadable.)

Category | Sub-category | Key variables | Names of variables
---|---|---|---
Admin (1 / 1) | Nil (1 / 1) | (0 / 0) | 
BackgroundHx (152 / 186) | Demographics (17 / 22) | (8 / 8) | 
 ( / ) | Drug history (106 / 106) | (26 / 26) | 
 ( / ) | Family history (8 / 20) | (5 / 6) | 
 ( / ) | Past medical history (21 / 38) | (5 / 5) | 
Blood test (366 / 2722) | Amino acids and related metabolites (49 / 210) | (0 / 0) | 
 ( / ) | Full blood count (0 / 71) | (0 / 0) | 
 ( / ) | Lipidomics (310 / 1575) | (8 / 8) | 
 ( / ) | Liver function (0 / 28) | (0 / 0) | 
 ( / ) | Neuronal pathology (1 / 4) | (1 / 4) | 
 ( / ) | Other metabolites (3 / 426) | (0 / 0) | 
 ( / ) | Pesticides (0 / 2) | (0 / 0) | 
 ( / ) | Proteomics (3 / 391) | (0 / 1) | 
 ( / ) | Purine metabolites (0 / 15) | (0 / 0) | 
CSF (11 / 1029) | Biochemistry (2 / 2) | (1 / 1) | 
 ( / ) | Cell count (2 / 4) | (1 / 1) | 
 ( / ) | Neuronal pathology (5 / 20) | (4 / 13) | 
 ( / ) | Proteomics (2 / 1003) | (1 / 1) | 
Clinical (195 / 552) | Adverse events (8 / 28) | (0 / 0) | 
 ( / ) | Diagnosis (12 / 39) | (5 / 5) | 
 ( / ) | Nil (0 / 1) | (0 / 0) | 
 ( / ) | Recruitment criteria (0 / 36) | (0 / 0) | 
 ( / ) | Symptoms - Related to cognition (114 / 366) | (13 / 13) | 
 ( / ) | Symptoms - Unlikely to be related to cognition (41 / 62) | (0 / 0) | 
 ( / ) | Vital signs (20 / 20) | (4 / 4) | 
Genetics (8 / 10) | APOE4 (2 / 2) | (2 / 2) | 
 ( / ) | Nil (4 / 6) | (0 / 0) | 
 ( / ) | Polygenic risk score (2 / 2) | (1 / 1) | 
Imaging (883 / 1924) | Amyloid PET (241 / 241) | (241 / 241) | 
 ( / ) | FDG PET (44 / 212) | (44 / 212) | 
 ( / ) | MRI - ASL (0 / 690) | (0 / 0) | 
 ( / ) | MRI - DTI (228 / 228) | (0 / 0) | 
 ( / ) | Others (3 / 15) | (0 / 0) | 
 ( / ) | Structural MRI - Stroke (2 / 5) | (2 / 5) | 
 ( / ) | Structural MRI - Volumetrics (349 / 351) | (349 / 351) | 
 ( / ) | Structural MRI - WMH (1 / 1) | (1 / 1) | 
 ( / ) | Tau PET (0 / 161) | (0 / 161) | 
 ( / ) | fMRI (15 / 20) | (0 / 0) | 
NPA (367 / 1254) | ADAS-Cog (127 / 400) | (1 / 1) | 
 ( / ) | ANART (1 / 52) | (1 / 1) | 
 ( / ) | AVLT (22 / 266) | (6 / 6) | 
 ( / ) | BNT (6 / 37) | (1 / 1) | 
 ( / ) | CBB (0 / 12) | (0 / 1) | 
 ( / ) | CCI (20 / 21) | (0 / 1) | 
 ( / ) | CDR (10 / 10) | (2 / 2) | 
 ( / ) | CDT (12 / 23) | (2 / 2) | 
 ( / ) | Category fluency (3 / 8) | (0 / 0) | 
 ( / ) | Digit span (0 / 32) | (0 / 4) | 
 ( / ) | FAQ (11 / 11) | (1 / 1) | 
 ( / ) | FCI (0 / 56) | (0 / 1) | 
 ( / ) | GDS (16 / 16) | (1 / 1) | 
 ( / ) | Logical Memory (4 / 10) | (2 / 2) | 
 ( / ) | MINT (0 / 3) | (0 / 0) | 
 ( / ) | MMSE (33 / 42) | (1 / 1) | 
 ( / ) | MOCA (43 / 43) | (1 / 1) | 
 ( / ) | MemTrax (0 / 21) | (0 / 0) | 
 ( / ) | NPI (36 / 152) | (1 / 1) | 
 ( / ) | NPIQ (13 / 25) | (1 / 1) | 
 ( / ) | Others (4 / 5) | (4 / 4) | 
 ( / ) | TMT (6 / 7) | (6 / 6) | 
 ( / ) | WAISR (0 / 2) | (0 / 1) | 
Other tests (0 / 506) | ECG (0 / 8) | (0 / 0) | 
 ( / ) | Histopathology (0 / 498) | (0 / 36) | 
Urine test (0 / 10) | Nil (0 / 10) | (0 / 0) | 
TOTAL (1983 / 8194) |  ( / ) | (753 / 1145) | 


In [19]:
import shelve
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

with shelve.open('data_availability_draft_1') as db:
    results_rid_num = db['results_rid_num'] # Numpy array
    results_var_num = db['results_var_num'] # Numpy array
    results_var_names = db['results_var_names'] # Pandas dataframe
    results_data_availability = db['results_data_availability'] # Numpy array
    cat_names = db['cat_names'] # Pandas dataframe

In [42]:
"""This script looks for a pair of [x,y] values, where x and y are the data availability thresholds for subjects and variables
respectively, such that exactly the same variables would be available for both [x,y] values. The user begins by specifying
the desired thresholds for subjects and data availability (by table cells, instead of rows/subjects or columns/variables), for
both the Main and Reduced Datasets. The script then computes [x,y] values that satisfy the user-specified thresholds. It then
looks for a pair of [x,y] values, comprising one value each for the Main and Reduced Datasets, that maximise the number of
variables available.

Data dependencies: Make sure that the following data variables have been imported from an appropriate shelf, such as
data_availability_draft_1.
- results_rid_num
- results_var_num
- results_var_names
- results_data_availability
"""

import numpy as np
import pandas as pd
import time

# Specify desired data availability thresholds for both Main and Reduced datasets. Each list can only contain 2 values. Follow
# the convention that the Main Dataset's thresholds are stated first, followed by the Reduced Dataset's.
rid_num_thresholds = [1000, 200]
results_data_availability_thresholds = [70, 90]

# Initialise results variables.
results = pd.DataFrame(
    index=['common_x_results', 'common_y_results', 'results_var_num_subset_results'],
    columns=range(0,2)
)

# Calculate the results.
for temp_idx, rid_num_threshold in enumerate(rid_num_thresholds):
    
    results_data_availability_threshold = results_data_availability_thresholds[temp_idx]

    results_rid_num_idxs = np.nonzero(results_rid_num >= rid_num_threshold)
    results_data_availability_idxs = np.nonzero(results_data_availability >= results_data_availability_threshold)

    idxs_shared_filter = np.zeros((len(results_rid_num_idxs[0])), dtype=bool)

    for x_idx, x in enumerate(results_rid_num_idxs[0]):
        y = results_rid_num_idxs[1][x_idx]
        if np.sum(results_data_availability_idxs[1][results_data_availability_idxs[0]==x]==y) > 0:
            idxs_shared_filter[x_idx] = True

    common_x = results_rid_num_idxs[0][idxs_shared_filter]
    common_y = results_rid_num_idxs[1][idxs_shared_filter]

    results_var_num_subset = np.zeros((len(common_x),))

    for x_idx, x in enumerate(common_x):
        y = common_y[x_idx]
        results_var_num_subset[x_idx] = results_var_num[x,y]
    
    results.loc['common_x_results', temp_idx] = common_x
    results.loc['common_y_results', temp_idx] = common_y
    results.loc['results_var_num_subset_results', temp_idx] = results_var_num_subset

# The numbers of variables available at each [x,y] value are stored in results_var_num_subset_results as Numpy arrays. Find
# the numbers that are common to both arrays. Then look for the highest of those numbers that correspond to exactly the same
# variables being available for both Main and Reduced Datasets.

# Numpy's intersect1d function returns an array sorted in ascending order.
common_var_nums = np.intersect1d(
    results.loc['results_var_num_subset_results',0],
    results.loc['results_var_num_subset_results',1]
)

for var_num in common_var_nums[-1::-1]: # common_var_nums is indexed such that it is examined in descending order.
    
    tic = time.perf_counter()
    
    temp_idxs_main = np.nonzero(results_var_num_subset_main==var_num)
    temp_idxs_reduced = np.nonzero(results_var_num_subset_reduced==var_num)
    
    # Initialise results variable. Columns 1 and 2 contain the subject and variable data availability thresholds for the
    # Main dataset. Columns 3 and 4 contain the same values, in that same order, for the Reduced Dataset.
    matches = np.empty((len(temp_idxs_main)*len(temp_idxs_reduced),4))
    matches[:] = np.nan
    matches_idx = 0
    
    for temp_idx_main in temp_idxs_main:
        x_idx_main = results.loc['common_x_results',0][temp_idx_main]
        y_idx_main = results.loc['common_y_results',0][temp_idx_main]
        var_names_main = set(results_var_names.iloc[x_idx_main,y_idx_main])
        
        for temp_idx_reduced in temp_idxs_reduced:
            x_idx_reduced = results.loc['common_x_results',1][temp_idx_reduced]
            y_idx_reduced = results.loc['common_y_results',1][temp_idx_reduced]
            var_names_reduced = set(results_var_names.iloc[x_idx_reduced,y_idx_reduced])
            
            if len(var_names_main & var_names_reduced)==var_num:
                matches[matches_idx,0] = x_idx_main
                matches[matches_idx,1] = y_idx_main
                matches[matches_idx,2] = x_idx_reduced
                matches[matches_idx,3] = y_idx_reduced
                matches_idx += 1
    
    toc = time.perf_counter()
    print("This loop took " + str(tic - toc) + " seconds.")
    
    if matches_idx > 0:
        break

matches = matches[:matches_idx,:] # Remove empty rows from matches.

if matches.size > 0:
    print("Matches were successfully found for a total number of variables of " + str(var_num) + ".")
else:
    print("No matches were found.")

In [78]:
# FOR TESTING CODE SNIPPETS
print("Matches were successfully found for a total number of variables of " + str(var_num) + ".")

Matches were successfully found for a total number of variables of 1.


In [76]:
# FOR TESTING CODE SNIPPETS
x.size > 0

True